In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset = pd.read_csv('/kaggle/input/on-time-graduation-classification/data_lulus_tepat_waktu.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
dataset.isnull().values.any()

In [ ]:
ket = {"Ya" : 0, "Tidak" : 1}
dataset["tepat"] = dataset["tepat"].map(ket)

In [ ]:
dataset['tepat'].value_counts()

In [ ]:
print('Ya', round(dataset['tepat'].value_counts()[0]/len(dataset) * 100,2), '% of the dataset')
print('Tidak', round(dataset['tepat'].value_counts()[1]/len(dataset) * 100,2), '% of the dataset')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

colors = ["#0101DF", "#DF0101"]

sns.countplot('tepat', data=dataset, palette=colors)
plt.title('Class Distributions \n (0: YA || 1: Tidak)', fontsize=14)

# Dataset tidak balance, jomplang bet perbedaannya.

kita Resampling datanya. Resampling ada 2 :


* Random Oversampling: Randomly duplicate examples in the minority class.
* Random Undersampling: Randomly delete examples in the majority class.

disini kita menggunakan random over sampling

[Referensi dari sini le, buat resampling nya](http://https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets/log)

In [ ]:
# Class count
count_class_0, count_class_1 = dataset.tepat.value_counts()

# Divide by class
df_class_0 = dataset[dataset['tepat'] == 0]
df_class_1 = dataset[dataset['tepat'] == 1]

In [ ]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.tepat.value_counts())

df_test_over.tepat.value_counts().plot(kind='bar', title='Count (tepat)');

In [ ]:
df_test_over.tepat.value_counts()

gas kan le, kalo udah balance

In [ ]:
x = df_test_over.iloc[:, :-1].values
y = df_test_over.iloc[:, -1].values

In [ ]:
from sklearn.model_selection import train_test_split

validation_size = 0.20
num_trees = 100
seed = 5
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=validation_size, random_state=seed)

In [ ]:
#Check Algorithms
models = []
models.append(( 'LR' , LogisticRegression()))
models.append(( 'LDA' , LinearDiscriminantAnalysis()))
models.append(( 'KNN' , KNeighborsClassifier()))
models.append(( 'CART' , DecisionTreeClassifier()))
models.append(( 'NB' , GaussianNB()))
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, random_state=seed)))
# evaluate each model
results = []
names = []
for name, model in models:
  kfold = KFold(n_splits=2, random_state=seed)
  cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring= 'accuracy' )
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

# **RF paling besar 94%**

In [ ]:
rf = RandomForestClassifier(n_estimators=num_trees, random_state=seed)
rf.fit(x_train, y_train)
predictions = rf.predict(x_test)
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

# kita test

In [ ]:
# ip1 = 3.17
# ip2 = 3.02
# ip3 = 3.28
# ip4 = 2.96

prediction_rf=rf.predict([[3.17,3.02,3.28,2.96]])
score1 = rf.score(x_test, y_test)
if prediction_rf[0] == 0:
    pred = "Tepat Waktu"
else:
    pred = "Tidak Tepat Waktu"
print('Prediksi :',pred)
print("Test score: {0:.2f} %".format(100 * score1)) 

In [ ]:
prediction_rf=rf.predict([[3.07,3.04,3.39,3.55]])
score1 = rf.score(x_test, y_test)
if prediction_rf[0] == 0:
    pred = "Tepat Waktu"
else:
    pred = "Tidak Tepat Waktu"
print('Prediksi :',pred)
print("Test score: {0:.2f} %".format(100 * score1)) 